In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

## gensim word2vec preparation

In [19]:
!pip install -U gensim
#we use this for the word2vec representations of each word
#remove if not using google colab. Instead, install from console

Requirement already up-to-date: gensim in /usr/local/lib/python3.6/dist-packages (3.4.0)
Requirement not upgraded as not directly required: scipy>=0.18.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (0.19.1)
Requirement not upgraded as not directly required: smart-open>=1.2.1 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.6.0)
Requirement not upgraded as not directly required: six>=1.5.0 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: numpy>=1.11.3 in /usr/local/lib/python3.6/dist-packages (from gensim) (1.14.5)
Requirement not upgraded as not directly required: boto3 in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (1.7.50)
Requirement not upgraded as not directly required: boto>=2.32 in /usr/local/lib/python3.6/dist-packages (from smart-open>=1.2.1->gensim) (2.48.0)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.6/dist-packages (fr

use gensim to get the word2vec representations of each word  
reference here:  
https://www.quora.com/How-do-you-extract-vectors-from-word2vec-given-a-word

In [0]:
from gensim.models import KeyedVectors#, Word2Vec

#currently using gensim's default word2vec pretrained model, trained on googlenews data. 
#todo: train on wikiqa if the results are crap. 

w2vmodel = KeyedVectors.load_word2vec_format('drive/Colab Notebooks/datasets/word2vec/GoogleNews-vectors-negative300-SLIM.bin.gz', binary=True)

word2vec first testing

In [2]:

# print(w2vmodel.similarity('woman','man'))
wvdict = w2vmodel.wv.vocab
#this is the list of all the words in the vocabulary

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [40]:
print(w2vmodel.index2word[0])

in


## Preprocessing input strings

In [0]:
import numpy as np

`a.toks` contains the questions (repeat question lines correspond to the number of candidate answer sentences)  
`b.toks` contains the answers. Each line is the answer to the question on the same line of `a.toks`  
`id` contains the id of the question at that line in `a.toks`  
`sim` contains the labels: for each block of repeated-questions: there is one line containing a `1` corresponding to the line in `b.toks` containing the correct answer to the same line in `a.toks`

https://github.com/castorini/data/tree/master/WikiQA

In [0]:
def read_data(path, readtype):
  with open(path) as f:
    lines = f.readlines()
    
  if readtype == "questions" or readtype == "answers":
    return lines
  elif readtype == "labels":
    return [int(line) for line in lines]

Load the data

In [0]:
bigpath = "drive/Colab Notebooks/datasets/QuestionAnswering/wikiQA"
questions = read_data(bigpath+"/train/a.toks", "questions")
answers = read_data(bigpath+"/train/b.toks", "answers")
labels = read_data(bigpath+"/train/sim.txt", "labels")
#question_ids = read_data(bigpath+"/train/id.txt", "labels")
#questions

In [24]:
print(questions[50])
print(answers[50])

how did anne frank die

born a german national , frank lost her citizenship in 1941 .



#### preparing word-level sequence embedding

This is to prepare for the Embedding layer

1. concatenate questions and answers into a list of `[question | answer]` pairs. This is a list of sequences.  
2. find max sequence length from this list  
3. the vocab size of the google news word2vec is 3 million words, the number of features is 300

The embedding_matrix:
1. let each word in `w2vmodel.wv.vocab.keys()` have an index
2. concatenate axis=1 all the word vectors in the vocabulary into embedding_matrix
3. then turn each string sequence into an array of numbers (each number represents the index of that word in the vocab)
4. pass the string sequences into the .fit as training input. The shape of this input should be (n_seqs, max_seq_length, n_features)
5. the embedding matrix will be the weights in the Embedding layer

In [0]:
vocab_size = 299567+1 #this is the SLIMmed googlenews w2v model's vocab count. #the last item is the 'all-zeroes' word vector, used for words that aren't in the w2vmodel

In [8]:
#make embedding matrix
embedding_matrix = np.zeros((vocab_size, 300))
for i in range(len(wvdict)):
    embedding_vector = w2vmodel.wv[w2vmodel.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [0]:
embedding_matrix[299566]

#### define some functions to preprocess data (both train and test)

In [0]:
def SEQ_max_length(sequences):
  return max( [len(qa.split()) for qa in sequences] )

In [12]:
#used for versions 0
def SEQ_make_qa_sequences_list(questions, answers):
  #remove question marks if there is one
  questions = [qu.replace(' ?', '') for qu in questions]
  #concatenate question and answer
  sequences = [qu+' '+ans for qu, ans in zip(questions, answers)]
  
  return sequences

# def STR_purge(s):
#   s = ''.join(i for i in s if not i.isdigit())
#   s = ''.join(i for i in s if not (i in ['of']))
#   return s

# questions = [qu.replace(' ?', '') for qu in questions]
# questions = [STR_purge(qu) for qu in questions]

#for version 0
#sequences = SEQ_make_qa_sequences_list(questions, answers)
#max_seq_length = SEQ_max_length(sequences)


In [0]:
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [0]:
def SEQ_token_and_pad(sequences, max_seq_length):
  
#   #version 0a
#   t = Tokenizer()
#   t.fit_on_texts(sequences)

#   vocab_size = len(t.word_index)+1

#   encoded_seqs = t.texts_to_sequences(sequences)
#   print(encoded_seqs)

#   #version 0b and up
  encoded_seqs = []
  for qa in sequences:
    encoded_qa = []
    tokenized_qa = qa.split()
    for word in tokenized_qa:
      index_to_append = vocab_size - 1
      
      if word in wvdict.keys():
        index_to_append = w2vmodel.vocab[word].index
      encoded_qa.append(index_to_append)
    
    encoded_seqs.append(encoded_qa)
    
  print(encoded_seqs)
  
  padded_seqs = pad_sequences(encoded_seqs, maxlen = max_seq_length, padding="post", value = vocab_size - 1)
  return padded_seqs

#version 0
#padded_seqs = SEQ_token_and_pad(sequences, max_seq_length)


Performing the necessary preprocessing on our data using our functions

In [12]:

max_question_length = SEQ_max_length(questions)
max_answer_length = SEQ_max_length(answers)
max_hidden_length = max(max_question_length, max_answer_length)

#version 1: getting separate questions and answers vectors (but still encoded in index form, not in pure word vector form)
padded_questions = SEQ_token_and_pad(questions, max_hidden_length)
padded_answers = SEQ_token_and_pad(answers, max_hidden_length)


[[132, 17, 27136, 22292, 2874, 299567], [132, 17, 27136, 22292, 2874, 299567], [132, 17, 27136, 22292, 2874, 299567], [132, 17, 27136, 22292, 2874, 299567], [132, 17, 27136, 22292, 2874, 299567], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 3, 299567, 37654, 299567, 369], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17, 9, 94046, 51278, 2508, 1032], [132, 141, 17,

[[299567, 4602, 16068, 27136, 12819, 4, 299567, 255051, 27136, 299567], [9, 2205, 19554, 3, 1254, 299567, 6638, 137], [2205, 20419, 0, 9, 299567, 27136, 12819], [299567, 27136, 12819, 3, 299567, 12819, 2874, 380, 9, 2205, 299567, 299567, 27136, 299567], [27136, 22292, 17, 608, 250, 2205, 22292, 299567, 30, 25, 460, 3, 3455, 219, 299567, 4480, 24223, 22292, 2, 3052, 299567, 2205, 299567], [25, 37654, 22, 299567, 1479, 299567, 41, 299567, 38212, 299567], [8748, 36943], [0, 9, 154, 299567, 1244, 299567, 61962, 299567, 299567, 37654, 3, 9, 548, 1426, 299567, 25515, 219, 1, 3658, 15, 299567, 4541, 299567], [0, 9, 46492, 299567, 76723, 299567, 100, 19708, 496, 299567, 299567, 37654, 3, 299567, 1426, 299567, 527, 25515, 1536, 219, 1, 1654, 299567], [0, 496, 111, 299567, 37654, 3, 299567, 1654, 25515, 299567, 9, 10026, 3851, 299567, 9, 154, 37654, 3, 768, 9, 12198, 25515, 26, 9, 11770, 25515, 299567], [299567, 195530, 299567, 36821, 9, 1479, 299567, 40, 37654, 299567, 3, 6936, 219, 12, 299567,

In [35]:
#rebuild a question and answer from the padded representation
print(padded_questions[0])
print(max_hidden_length)

" ".join([w2vmodel.wv.index2word[index] for index in padded_answers[0] if not index == 299567])



[   132     17  27136  22292   2874 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567
 299567 299567 299567 299567 299567 299567 299567 299567 299567 299567]
40


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  after removing the cwd from sys.path.


'partly submerged glacier cave on moreno glacier'

#### creating one-hot labels

In [0]:
from sklearn.preprocessing import OneHotEncoder

In [0]:
labels_np = np.array(labels)
labels_np = np.expand_dims(labels_np, axis=1)

onehot = OneHotEncoder(n_values = 2)
onehot.fit(labels_np)
labels_np = onehot.transform(labels_np)

## Model building and training

### Guesswork and experimentation



---


*If i'm doing the implementation in the 1506.06490 paper*  
**Todo**: 

- find length of the longest question and longest answer
- use these maxes for padding to shorter sentences
- in one big model, build 2 separate convnet submodels, one for questions, one for answers
- concatenate the outputs for the questions and answers convnets, and we obtain the **joint representation** `p` of each question-answer pair.

Each of these learned joint pairs is a single **timestep** in the LSTM part. We learn the sequence and semantic relationships between each answer-candidate sentence in the context paragraph, not between the words themselves. (the word-level patterns are learned in the Conv stage already)

for the LSTM stage, padding instead involves adding dummy qa pairs for each timestep without an answer. The max number of answer candidates in the dataset (and possible qa pairs, per question) is 30, so any Qs with fewer than 30 answer sentences will be padded with dummy pairs.


---



*my own simple-ass idea*  
**Todo:**

version 0a: `shit lstm model copped from some blog`  
https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

version 0b: `with learning rate, and fixed word tokenizations`

version 1a: `separate LSTM for question and answer, then Concatenate into a hidden layer which then is fed into classifier/Dense/whatever`

version 1b: `no LSTM for question and answer. Using Functional API. WORKS but OVERFITS.`

### Giang's idea

Giang's idea:

`doc question  
input_1 = Input(...)  
...  
output_1 = ...


doc answer  
input_2 = Input(...)  
...  
output_2 = ...

input_2 = Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_answer_length, trainable=False)  
  
output_2 = LSTM(100)(input_reshape_2)

hidden = Concatenate()([output_1, output_2])
classifier = Dense(1, activation="sigmoid")(hidden)
`

In [0]:
# #not using functional API
# #dont use this, non working

# questions_model = Sequential()
# questions_model.add(equestions)
# #questions_model.add(LSTM(64))
# # questions_model.add(Activation('softmax'))
# #questions_model.add(Dropout(0.4))

# answers_model = Sequential()
# answers_model.add(eanswers)
# #answers_model.add(LSTM(64))
# # answers_model.add(Activation('softmax'))
# #answers_model.add(Dropout(0.4))

# merged = Merge([questions_model.output, answers_model.output], mode = "concat")
# print(merged.output.shape)
# final_model = Sequential()
# final_model.add(merged)
# #final_model.add(Flatten())
# final_model.add(LSTM(100)) #comment out if using lstm on the question and answer submodels
# final_model.add(Dense(2, activation="softmax"))

# final_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])
# print(final_model.summary())

In [31]:
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, LSTM, Dense
from keras.optimizers import Adam

#using keras functional api

qumodel_input = Input(shape = (max_hidden_length,), name="questionsMainInput")
qumodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(qumodel_input)
print(qumodel_embed.shape)

ansmodel_input = Input(shape = (max_hidden_length,), name="answersMainInput")
ansmodel_embed = Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_hidden_length, trainable=False)(ansmodel_input)

combinedmodel = Concatenate()([qumodel_embed, ansmodel_embed])
print(combinedmodel.shape)

lstmcombinedmodel = LSTM(100)(combinedmodel)
classifier = Dense(2, activation='softmax')(lstmcombinedmodel)

final_model = Model(inputs = [qumodel_input, ansmodel_input], outputs = classifier)

final_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(), metrics=['accuracy'])
print(final_model.summary())

(?, 40, 300)
(?, 40, 600)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
questionsMainInput (InputLayer) (None, 40)           0                                            
__________________________________________________________________________________________________
answersMainInput (InputLayer)   (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 40, 300)      89870400    questionsMainInput[0][0]         
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 40, 300)      89870400    answersMainInput[0][0]           
___________________________________________________________________________________

### Attempting the 1506 06490 paper

In [22]:
from keras.layers import Dense, LSTM, Conv1D, Embedding, MaxPooling1D, Activation, Merge, Flatten
from keras.models import Sequential
from keras.optimizers import Adadelta

In [25]:
#embedding is Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False)

questions_model = Sequential()
questions_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
questions_model.add(Conv1D(100, 5, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=2))
questions_model.add(Activation('relu'))

questions_model.add(Conv1D(100, 3, strides=1))
questions_model.add(MaxPooling1D(pool_size=1))
questions_model.add(Activation('relu'))

answers_model = Sequential()
answers_model.add(Embedding(vocab_size, 300, weights = [embedding_matrix], input_length = max_hidden_length, trainable = False))
answers_model.add(Conv1D(100, 5, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=2))
answers_model.add(Activation('relu'))

answers_model.add(Conv1D(100, 3, strides=1))
answers_model.add(MaxPooling1D(pool_size=1))
answers_model.add(Activation('relu'))

merge = Merge([questions_model, answers_model])

print(merge.output.shape)

final_model = Sequential()
final_model.add(merge)
final_model.add(Flatten())
# final_model.add(LSTM(160))
final_model.add(Dense(1, activation="softmax"))

final_model.compile(loss = 'binary_crossentropy', optimizer = Adadelta(lr=0.5), metrics=['accuracy'])
print(final_model.summary())


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


(?, 6, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 6, 100)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 600)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 601       
Total params: 180,162,001
Trainable params: 421,201
Non-trainable params: 179,740,800
_________________________________________________________________
None


### Attempting the dl-models-for-qa QA-LSTM model

In [46]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Merge, Dense
WORD2VEC_EMBED_SIZE = 300
QA_EMBED_SIZE = 64
BATCH_SIZE = 64
NBR_EPOCHS = 10

qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_matrix], mask_zero=True))
qenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, input_shape=(None, 300, max_hidden_length), 
                            return_sequences=False), merge_mode="sum"))
qenc.add(Dropout(0.3))

aenc = Sequential()
aenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   weights=[embedding_matrix], mask_zero=True))
aenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, input_shape=(None, 300, max_hidden_length), 
                            return_sequences=False), merge_mode="sum"))
aenc.add(Dropout(0.3))

model = Sequential()
model.add(Merge([qenc, aenc], mode="sum"))
model.add(Dense(1, activation="softmax"))

model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=["accuracy"])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 180,114,625
Trainable params: 180,114,625
Non-trainable params: 0
_________________________________________________________________
None


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


## Fit the chosen model

Severely punish "no" answers because that's just lazy. The network should at least try to guess "yes" or "1" on at least some questions, and not settle for a 88% accuracy by answering no to everything.

add class_weights and try:

In [0]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight('balanced', np.unique(labels), labels)
#class_weights = np.array([0.5, 24.])

In [32]:
#final_model.fit([padded_questions, padded_answers], labels, epochs=5, batch_size=128, verbose=1, class_weight = class_weights, validation_split=0.2)

#model.fit([padded_questions, padded_answers], labels, batch_size=32,
 #         epochs=20, validation_split=0.1, class_weight = class_weights)
  
final_model.fit([padded_questions, padded_answers], labels_np, epochs=30, batch_size=64, verbose=1, validation_split=0.1)

Train on 7804 samples, validate on 868 samples
Epoch 1/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.3949 - acc: 0.8765 - val_loss: 0.3631 - val_acc: 0.8698
Epoch 2/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.3258 - acc: 0.8798 - val_loss: 0.3378 - val_acc: 0.8698
Epoch 3/30
7680/7804 [============================>.] - ETA: 0s - loss: 0.3179 - acc: 0.8820

7804/7804 [==============================] - 14s 2ms/step - loss: 0.3186 - acc: 0.8817 - val_loss: 0.3315 - val_acc: 0.8687
Epoch 4/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.3149 - acc: 0.8820 - val_loss: 0.3574 - val_acc: 0.8733
Epoch 5/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.3109 - acc: 0.8835 - val_loss: 0.3388 - val_acc: 0.8721
Epoch 6/30
3968/7804 [==============>...............] - ETA: 6s - loss: 0.3003 - acc: 0.8851

7804/7804 [==============================] - 14s 2ms/step - loss: 0.2977 - acc: 0.8874 - val_loss: 0.3300 - val_acc: 0.8664
Epoch 7/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2948 - acc: 0.8897 - val_loss: 0.3326 - val_acc: 0.8721
Epoch 8/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2919 - acc: 0.8856 - val_loss: 0.3274 - val_acc: 0.8641
Epoch 9/30
3328/7804 [===========>..................] - ETA: 7s - loss: 0.2832 - acc: 0.8975

7804/7804 [==============================] - 14s 2ms/step - loss: 0.2830 - acc: 0.8926 - val_loss: 0.3395 - val_acc: 0.8479
Epoch 10/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2770 - acc: 0.8976 - val_loss: 0.3352 - val_acc: 0.8664
Epoch 11/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2752 - acc: 0.8986 - val_loss: 0.3835 - val_acc: 0.8618
Epoch 12/30
3200/7804 [===========>..................] - ETA: 7s - loss: 0.2623 - acc: 0.9062

7804/7804 [==============================] - 14s 2ms/step - loss: 0.2650 - acc: 0.9057 - val_loss: 0.3685 - val_acc: 0.8514
Epoch 13/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2616 - acc: 0.9030 - val_loss: 0.3723 - val_acc: 0.8571
Epoch 14/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2513 - acc: 0.9081 - val_loss: 0.4127 - val_acc: 0.8353
Epoch 15/30
3136/7804 [===========>..................] - ETA: 7s - loss: 0.2496 - acc: 0.9136

...

7804/7804 [==============================] - 14s 2ms/step - loss: 0.2330 - acc: 0.9170 - val_loss: 0.3578 - val_acc: 0.8618
Epoch 19/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2106 - acc: 0.9268 - val_loss: 0.4357 - val_acc: 0.8618
Epoch 20/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.2069 - acc: 0.9254 - val_loss: 0.4226 - val_acc: 0.8387
Epoch 21/30
3136/7804 [===========>..................] - ETA: 7s - loss: 0.2002 - acc: 0.9292

7804/7804 [==============================] - 14s 2ms/step - loss: 0.2101 - acc: 0.9256 - val_loss: 0.4431 - val_acc: 0.8237
Epoch 22/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1811 - acc: 0.9343 - val_loss: 0.5342 - val_acc: 0.8203
Epoch 23/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1727 - acc: 0.9387 - val_loss: 0.4362 - val_acc: 0.8525
Epoch 24/30
3136/7804 [===========>..................] - ETA: 7s - loss: 0.1574 - acc: 0.9445

7804/7804 [==============================] - 14s 2ms/step - loss: 0.1565 - acc: 0.9463 - val_loss: 0.5255 - val_acc: 0.8203
Epoch 25/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1488 - acc: 0.9476 - val_loss: 0.4818 - val_acc: 0.8191
Epoch 26/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1524 - acc: 0.9476 - val_loss: 0.4628 - val_acc: 0.8318
Epoch 27/30
3136/7804 [===========>..................] - ETA: 7s - loss: 0.1349 - acc: 0.9589

7804/7804 [==============================] - 14s 2ms/step - loss: 0.1377 - acc: 0.9562 - val_loss: 0.5794 - val_acc: 0.8134
Epoch 28/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1257 - acc: 0.9607 - val_loss: 0.5314 - val_acc: 0.8099
Epoch 29/30
7804/7804 [==============================] - 14s 2ms/step - loss: 0.1331 - acc: 0.9540 - val_loss: 0.5469 - val_acc: 0.8491
Epoch 30/30
3136/7804 [===========>..................] - ETA: 7s - loss: 0.1128 - acc: 0.9665

7804/7804 [==============================] - 14s 2ms/step - loss: 0.1180 - acc: 0.9621 - val_loss: 0.5348 - val_acc: 0.8214


## Testing the trained model

#### Preparing the test data

In [0]:
test_questions = ["Where was Tuan born","Where was Tuan born","Where was Tuan born","Where was Tuan born"]
test_answers = ["Pigs dogs cats","Boo","Candy balls cake", "Tuan was born in Vietnam"]
test_labels = [0, 0, 0, 1]



# test_questions = read_data(bigpath+"/test/a.toks", "questions")
# test_answers = read_data(bigpath+"/test/b.toks", "answers")
# test_labels = read_data(bigpath+"/test/sim.txt", "labels")
# question_ids = read_data(bigpath+"/dev/id.txt", "labels")

In [34]:
#test_seqs, max_test_seq_length = SEQ_make_qa_sequences_list(test_questions, test_answers)
padded_test_questions = SEQ_token_and_pad(test_questions, max_hidden_length)
padded_test_answers = SEQ_token_and_pad(test_answers, max_hidden_length)

[[3536, 8, 65893, 1550], [3536, 8, 65893, 1550], [3536, 8, 65893, 1550], [3536, 8, 65893, 1550]]
[[37419, 2691, 6870], [32824], [19906, 3780, 8051], [65893, 8, 1550, 0, 3508]]


In [32]:
padded_seqs.shape


NameError: ignored

#### Predicting some q, a pairs

In [0]:
test_preds = final_model.predict_on_batch([padded_test_questions, padded_test_answers])

In [36]:
print(test_preds)
np.argmax(test_preds*1000)

[[0.9929662  0.00703384]
 [0.9932609  0.00673912]
 [0.9931613  0.00683869]
 [0.993171   0.00682908]]


2

In [30]:
i=2 #test example to check

# print(padded_test_seqs[1])
# print(padded_test_seqs[16])

print(test_preds[i])
print(test_questions[i])
print(test_answers[i])
print("correct label: "+str(test_labels[i]))

[0.99620086 0.00379917]
how african americans were immigrated to the us

from the immigration and nationality act of 1965 to 2007 , an estimated total of 0.8 to 0.9 million africans immigrated to the united states , accounting for roughly 3.3 % of total immigration to the united states during this

correct label: 0
